We will use a Sea Surface Temperature (SST)

look for "SST Global grilled Data"
data are available on different web pages:
https://psl.noaa.gov/data/gridded/
 - 1Â° resolution
 - monthly data
 - satellite period (>1980)
 - mix all data

https://psl.noaa.gov/data/gridded/data.noaa.oisst.v2.html

# Import needed Libraries

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import hvplot.xarray

# xarray basic manipulation:
 - Open file
 - explore [Dataset](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.html)
 - explore [DataArray](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.html)

In [ ]:
data = xr.open_dataset('sst.mnmean.nc')

In [ ]:
data  # file Dataset

In [ ]:
data.data_vars

In [ ]:
data.coords

In [ ]:
data.sst  # sst DataArray http://xarray.pydata.org/en/stable/generated/xarray.DataArray.html

In [ ]:
print(data.sst.shape)
print(data.sst.ndim)
print(data.sst.dims)

In [ ]:
data.time.size

data index selection: **[isel()](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.isel.html)**  
data value selection: **[sel()](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.sel.html)**  
the **[slice](https://docs.python.org/3.8/library/functions.html#slice)** object  
the **[GroupBy](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.groupby.html)** object  

In [ ]:
data.sst.isel(lon=100, lat=50, time=0)     # select the point defined by its indexes (i=100, j=50, t=0)

In [ ]:
data.sst.sel(lon=180.5, lat=0.5)   # time series for a point defined by its location (lon = 180.5, lat = 0.5)

In [ ]:
data.sst.sel(lon=180.5, lat=slice(0.5,-0.5))  # Warning : respect latitude order 0.5 before -0.5

In [ ]:
print( data.sst.isel(time=200).shape )
print( data.sst.sel(time='2000-01-01T00:00:00.000000000').shape )
print( data.sst.sel(time='2000-01-01').shape )
print( data.sst.sel(time='2000-01').shape )
print( data.sst.sel(time='2000').shape )
print( data.sst.sel(time=slice('2000','2002')).shape )

In [ ]:
data.sst.isel(time=200).sel(lon=180.5, lat=slice(0.5,-0.5))

In [ ]:
data.sst.groupby('time.month').mean(dim='time').shape

In [ ]:
data = data.sel(time=slice('1982','2013'))   # keep only "complete" years

# a few basic plots
https://matplotlib.org/tutorials/colors/colormaps.html

In [ ]:
fig, axes = plt.subplots(2,1)
data.sst.isel(lon=100, lat=50).plot(ax=axes[0])
data.sst.sel(lon=180.5, lat=0.5).plot(ax=axes[1])
plt.show()

In [ ]:
mask = xr.open_dataset('lsmask.nc')
mask = mask.squeeze()  # get rid of the time dimension in the variable mask.mask
mask

In [ ]:
fig, axes = plt.subplots(2,1)
sst2d = data.sst.isel(time=100)
sst2d.plot(ax=axes[0])
sst2d = sst2d.where(mask.mask == 1.)
sst2d.plot(vmin=-2, vmax=32, cmap='gist_rainbow_r',ax=axes[1])
plt.show()

In [ ]:
p = data.sst.isel(time=200).where(mask.mask == 1.).plot(
    subplot_kws=dict( projection=ccrs.PlateCarree(central_longitude=200), facecolor='gray'),
    transform=ccrs.PlateCarree(),vmin=-2, vmax=32, cmap='gist_rainbow_r'
)
p.axes.set_global()
p.axes.coastlines()  

In [ ]:

#proj = ccrs.PlateCarree(central_longitude=200)
proj = ccrs.LambertCylindrical(central_longitude=200)

data.sst.isel(time=slice(1, 13)).where(mask.mask == 1.).hvplot.quadmesh(
    'lon', 'lat', clim=(-2,32), projection=proj, project=True, global_extent=True, 
    cmap='gist_rainbow_r', rasterize=True, dynamic=False, coastline=True, 
    frame_width=500, widget_location='bottom', widget_type='scrubber')

In [ ]:
data.sst.sel(lat=slice(1.5,-1.5)).mean(dim='lat').plot()

In [ ]:
data.sst.groupby('time.month').mean(dim='time').mean(dim=('lon','lat')).plot()

# basic computation:

Earth is not flat: 1Â°x1Â° grid does not have a constant area  
weight data with mask * cos( lat )

In [ ]:
weights = np.cos(np.deg2rad(mask.lat))
weights = mask*weights
weights

In [ ]:
fig, axes = plt.subplots(2,1)
mask.mask.plot(ax=axes[0])
weights.mask.plot(ax=axes[1])

In [ ]:
data.sst.mean(("lon", "lat")).plot(label="unweighted")
weights = np.cos(np.deg2rad(data.lat))
data.sst.weighted(weights).mean(("lon", "lat")).plot(label="weighted")
weights = np.cos(np.deg2rad(mask.lat))
weights = mask*weights
data.sst.weighted(weights.mask).mean(("lon", "lat")).plot(label="weighted+mask")
plt.legend()

In [ ]:
a = data.sst.groupby('time.month').mean(dim='time')
#a.mean(dim=('lon','lat')).plot(label="unweighted")
fig, axes = plt.subplots(3,1,figsize=(10, 10))
a.weighted(weights.mask).mean(dim=('lon','lat')).plot(label="gobal mean", ax=axes[0])
axes[0].set_title('gobal mean')
a.sel(lon=150.5, lat=2.5).plot(ax=axes[1])
axes[1].set_title('West Eq. Pac.')
a.sel(lon=300.5, lat=45.5).plot(ax=axes[2])
axes[2].set_title('North-West Atl.')

In [ ]:
a = data.sst.weighted(weights.mask).mean(("lon", "lat"))
a.plot(label="weighted+mask")
b = a.rolling(time=12, center=True).mean()
b.plot(label="rolling 1 year")
b = a.rolling(time=12*5, center=True).mean()
b.plot(label="rolling 5 years")
plt.legend()
c = a.rolling(time=12*10, center=True).mean()
c.plot(label="rolling 10 years")
plt.legend()

In [ ]:
a = data.sst.groupby('time.year').mean(dim='time')   # yearly mean
a.weighted(weights.mask).mean(dim=('lon','lat')).plot(label="weighted+mask")
#a.mean(dim=('lon','lat')).plot(label="unweighted")
plt.legend()

In [ ]:
a = data.sst.weighted(weights.mask).mean(dim=('lon','lat'))
clim = a.groupby("time.month").mean("time")
anom = a.groupby("time.month") - clim
anom.plot()

In [ ]:
a = data.sst.std(dim='time').where(mask.mask == 1.)
fig, axes = plt.subplots(2,1)
a.plot(vmin=0,vmax=6,ax=axes[0])
a.sel(lon=slice(260.5,320.5),lat=slice(50.5,10.5)).plot(vmin=0,vmax=6,ax=axes[1])

In [ ]:
a = data.sst.groupby('time.month').mean(dim='time')
#(a.max(dim='month') - a.min(dim='month')).where(mask.mask == 1.).plot(vmin=0,vmax=12)
a.std(dim='month').where(mask.mask == 1.).plot(vmin=0,vmax=6)

In [ ]:
a = data.sst.groupby("time.month")
anom = a - a.mean("time")
anom.std(dim='time').where(mask.mask == 1.).plot(vmin=0,vmax=2,cmap='YlGnBu')

In [ ]:
linfit = data.sst.polyfit('time', 1)
trend = xr.polyval(coord=data.time, coeffs=linfit.polyfit_coefficients)

linfit.polyfit_coefficients.isel(degree=1).where(mask.mask == 1.).plot()

In [ ]:
a = data.sst.groupby('time.year').mean(dim='time')   # yearly mean
a.weighted(weights.mask).mean(dim=('lon','lat')).plot(label="org")
a = trend.groupby('time.year').mean(dim='time')   # yearly mean
a.weighted(weights.mask).mean(dim=('lon','lat')).plot(label="trend")
plt.legend()

In [ ]:
sst_detrend = data.sst - trend.values
data['sst'] = sst_detrend
a = data.sst.groupby('time.year').mean(dim='time')   # yearly mean
a.weighted(weights.mask).mean(dim=('lon','lat')).plot(label="org")

In [ ]:
data.time.dt.days_in_month

In [ ]:
#data["time.season"] same as
data.time.dt.season

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
lon = (mask.lon * mask.mask).transpose()
lat = (mask.lat * mask.mask)
sst = data.sst.isel(time=200)
ax.scatter(lon, lat, sst)
plt.show()


In [ ]:
(mask.lat * mask.mask)